# Trading Behavior Analysis: Fear & Greed Index Impact
## Complete Analysis with Visualizations

In [ ]:
"""
Trading Behavior Analysis: Fear & Greed Index Impact
Standalone Python script version
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Create output directory
os.makedirs('outputs', exist_ok=True)

print("=" * 80)
print("TRADING BEHAVIOR ANALYSIS: FEAR & GREED INDEX IMPACT")
print("=" * 80)

## PART A: DATA PREPARATION

In [ ]:
print("\n" + "=" * 80)
print("PART A: DATA PREPARATION")
print("=" * 80)

# Load datasets
print("\nLoading datasets...")
fear_greed = pd.read_csv('fear_greed_index.csv')
trades = pd.read_csv('historical_data.csv')

print("\nFear & Greed Index Dataset:")
print(f"Rows: {len(fear_greed)}, Columns: {len(fear_greed.columns)}")
print(f"Columns: {list(fear_greed.columns)}")
print(f"\nMissing values:\n{fear_greed.isnull().sum()}")
print(f"\nDuplicates: {fear_greed.duplicated().sum()}")
print(f"\nDate range: {fear_greed['date'].min()} to {fear_greed['date'].max()}")

print("\n" + "-" * 80)
print("\nTrading Data:")
print(f"Rows: {len(trades)}, Columns: {len(trades.columns)}")
print(f"Columns: {list(trades.columns)}")
print(f"\nMissing values:\n{trades.isnull().sum()}")
print(f"\nDuplicates: {trades.duplicated().sum()}")
print(f"\nUnique accounts: {trades['Account'].nunique()}")

# Convert timestamps and align datasets
print("\nConverting timestamps and aligning datasets...")
fear_greed['date'] = pd.to_datetime(fear_greed['date'])
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], format='%d-%m-%Y %H:%M')
trades['date'] = trades['Timestamp IST'].dt.date
trades['date'] = pd.to_datetime(trades['date'])

print(f"Trading data date range: {trades['date'].min()} to {trades['date'].max()}")
print(f"Fear/Greed data date range: {fear_greed['date'].min()} to {fear_greed['date'].max()}")

# Merge datasets
trades_merged = trades.merge(fear_greed[['date', 'value', 'classification']], 
                              on='date', how='left')
trades_merged.rename(columns={'value': 'fg_value', 'classification': 'sentiment'}, inplace=True)

print(f"\nMerged dataset: {len(trades_merged)} rows")
print(f"Rows with sentiment data: {trades_merged['sentiment'].notna().sum()}")
print(f"\nSentiment distribution:")
print(trades_merged['sentiment'].value_counts())

# Create key metrics
print("\nCreating key metrics...")

# Calculate daily metrics per trader
daily_metrics = trades_merged.groupby(['Account', 'date']).agg({
    'Closed PnL': 'sum',
    'Size USD': ['sum', 'mean', 'count'],
    'Side': lambda x: (x == 'BUY').sum() / len(x),
    'sentiment': 'first',
    'fg_value': 'first'
}).reset_index()

daily_metrics.columns = ['Account', 'date', 'daily_pnl', 'total_volume', 
                         'avg_trade_size', 'num_trades', 'long_ratio', 'sentiment', 'fg_value']

# Calculate win rate
win_rate = trades_merged[trades_merged['Closed PnL'] != 0].groupby(['Account', 'date']).apply(
    lambda x: (x['Closed PnL'] > 0).sum() / len(x) if len(x) > 0 else 0
).reset_index(name='win_rate')

daily_metrics = daily_metrics.merge(win_rate, on=['Account', 'date'], how='left')

# Calculate leverage proxy
account_avg_size = trades_merged.groupby('Account')['Size USD'].mean()
trades_merged['leverage_proxy'] = trades_merged.apply(
    lambda x: x['Size USD'] / account_avg_size[x['Account']], axis=1
)

daily_leverage = trades_merged.groupby(['Account', 'date'])['leverage_proxy'].mean().reset_index()
daily_metrics = daily_metrics.merge(daily_leverage, on=['Account', 'date'], how='left')

# Calculate additional metrics
daily_metrics['short_ratio'] = 1 - daily_metrics['long_ratio']
daily_metrics = daily_metrics.sort_values(['Account', 'date'])
daily_metrics['cumulative_pnl'] = daily_metrics.groupby('Account')['daily_pnl'].cumsum()
daily_metrics['max_cumulative_pnl'] = daily_metrics.groupby('Account')['cumulative_pnl'].cummax()
daily_metrics['drawdown'] = daily_metrics['cumulative_pnl'] - daily_metrics['max_cumulative_pnl']

print(f"Daily metrics created: {daily_metrics.shape}")
print("\nSample metrics:")
print(daily_metrics.head(10))

## PART B: ANALYSIS

In [ ]:
print("\n" + "=" * 80)
print("PART B: ANALYSIS")
print("=" * 80)

# Question 1: Performance Differences - Fear vs Greed Days
print("\n" + "-" * 80)
print("QUESTION 1: PERFORMANCE DIFFERENCES - FEAR VS GREED DAYS")
print("-" * 80)

# Group sentiments
daily_metrics['sentiment_group'] = daily_metrics['sentiment'].map({
    'Extreme Fear': 'Fear',
    'Fear': 'Fear',
    'Neutral': 'Neutral',
    'Greed': 'Greed',
    'Extreme Greed': 'Greed'
})

# Compare performance metrics
performance_comparison = daily_metrics.groupby('sentiment_group').agg({
    'daily_pnl': ['mean', 'median', 'std'],
    'win_rate': 'mean',
    'drawdown': 'mean',
    'num_trades': 'mean'
}).round(2)

print("\nPerformance by Sentiment:")
print(performance_comparison)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

daily_metrics.boxplot(column='daily_pnl', by='sentiment_group', ax=axes[0, 0])
axes[0, 0].set_title('Daily PnL Distribution by Sentiment')
axes[0, 0].set_xlabel('Sentiment')
axes[0, 0].set_ylabel('Daily PnL (USD)')

sentiment_win_rate = daily_metrics.groupby('sentiment_group')['win_rate'].mean()
sentiment_win_rate.plot(kind='bar', ax=axes[0, 1], color=['red', 'gray', 'green'])
axes[0, 1].set_title('Average Win Rate by Sentiment')
axes[0, 1].set_ylabel('Win Rate')
axes[0, 1].set_xlabel('Sentiment')

daily_metrics.boxplot(column='drawdown', by='sentiment_group', ax=axes[1, 0])
axes[1, 0].set_title('Drawdown Distribution by Sentiment')
axes[1, 0].set_xlabel('Sentiment')
axes[1, 0].set_ylabel('Drawdown (USD)')

sentiment_trades = daily_metrics.groupby('sentiment_group')['num_trades'].mean()
sentiment_trades.plot(kind='bar', ax=axes[1, 1], color=['red', 'gray', 'green'])
axes[1, 1].set_title('Average Number of Trades by Sentiment')
axes[1, 1].set_ylabel('Number of Trades')
axes[1, 1].set_xlabel('Sentiment')

plt.tight_layout()
plt.savefig('outputs/performance_by_sentiment.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/performance_by_sentiment.png")

# Question 2: Behavioral Changes
print("\n" + "-" * 80)
print("QUESTION 2: BEHAVIORAL CHANGES BASED ON SENTIMENT")
print("-" * 80)

behavior_comparison = daily_metrics.groupby('sentiment_group').agg({
    'num_trades': ['mean', 'std'],
    'leverage_proxy': ['mean', 'std'],
    'long_ratio': 'mean',
    'short_ratio': 'mean',
    'avg_trade_size': ['mean', 'std']
}).round(3)

print("\nBehavioral Metrics by Sentiment:")
print(behavior_comparison)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

daily_metrics.boxplot(column='num_trades', by='sentiment_group', ax=axes[0, 0])
axes[0, 0].set_title('Trade Frequency by Sentiment')
axes[0, 0].set_ylabel('Number of Trades per Day')

daily_metrics.boxplot(column='leverage_proxy', by='sentiment_group', ax=axes[0, 1])
axes[0, 1].set_title('Leverage Usage by Sentiment')
axes[0, 1].set_ylabel('Leverage Proxy')

long_short_data = daily_metrics.groupby('sentiment_group')[['long_ratio', 'short_ratio']].mean()
long_short_data.plot(kind='bar', ax=axes[1, 0], stacked=True)
axes[1, 0].set_title('Long/Short Ratio by Sentiment')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend(['Long', 'Short'])

daily_metrics.boxplot(column='avg_trade_size', by='sentiment_group', ax=axes[1, 1])
axes[1, 1].set_title('Average Trade Size by Sentiment')
axes[1, 1].set_ylabel('Trade Size (USD)')

plt.tight_layout()
plt.savefig('outputs/behavior_by_sentiment.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/behavior_by_sentiment.png")

# Question 3: Trader Segmentation
print("\n" + "-" * 80)
print("QUESTION 3: TRADER SEGMENTATION")
print("-" * 80)

# Calculate trader-level statistics
trader_stats = daily_metrics.groupby('Account').agg({
    'daily_pnl': ['mean', 'std', 'sum'],
    'num_trades': 'mean',
    'leverage_proxy': 'mean',
    'win_rate': 'mean'
}).reset_index()

trader_stats.columns = ['Account', 'avg_daily_pnl', 'pnl_volatility', 'total_pnl', 
                        'avg_trades_per_day', 'avg_leverage', 'avg_win_rate']

# Create segments
leverage_median = trader_stats['avg_leverage'].median()
trader_stats['leverage_segment'] = trader_stats['avg_leverage'].apply(
    lambda x: 'High Leverage' if x > leverage_median else 'Low Leverage'
)

trades_median = trader_stats['avg_trades_per_day'].median()
trader_stats['frequency_segment'] = trader_stats['avg_trades_per_day'].apply(
    lambda x: 'Frequent' if x > trades_median else 'Infrequent'
)

trader_stats['consistency_score'] = trader_stats['avg_daily_pnl'] / (trader_stats['pnl_volatility'] + 1)
consistency_median = trader_stats['consistency_score'].median()
trader_stats['consistency_segment'] = trader_stats['consistency_score'].apply(
    lambda x: 'Consistent Winner' if x > consistency_median else 'Inconsistent'
)

print("\nTrader Segments:")
print(f"\nLeverage Segments:")
print(trader_stats['leverage_segment'].value_counts())
print(f"\nFrequency Segments:")
print(trader_stats['frequency_segment'].value_counts())
print(f"\nConsistency Segments:")
print(trader_stats['consistency_segment'].value_counts())

# Analyze segment performance
print("\n=== SEGMENT 1: High vs Low Leverage ===")
leverage_performance = trader_stats.groupby('leverage_segment').agg({
    'total_pnl': 'mean',
    'avg_win_rate': 'mean',
    'pnl_volatility': 'mean'
}).round(2)
print(leverage_performance)

print("\n=== SEGMENT 2: Frequent vs Infrequent Traders ===")
frequency_performance = trader_stats.groupby('frequency_segment').agg({
    'total_pnl': 'mean',
    'avg_win_rate': 'mean',
    'pnl_volatility': 'mean'
}).round(2)
print(frequency_performance)

print("\n=== SEGMENT 3: Consistent vs Inconsistent Winners ===")
consistency_performance = trader_stats.groupby('consistency_segment').agg({
    'total_pnl': 'mean',
    'avg_win_rate': 'mean',
    'pnl_volatility': 'mean'
}).round(2)
print(consistency_performance)

# Merge segments back
daily_metrics = daily_metrics.merge(trader_stats[['Account', 'leverage_segment', 
                                                   'frequency_segment', 'consistency_segment']], 
                                    on='Account', how='left')

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

trader_stats.boxplot(column='total_pnl', by='leverage_segment', ax=axes[0, 0])
axes[0, 0].set_title('Total PnL by Leverage Segment')
axes[0, 0].set_ylabel('Total PnL (USD)')

trader_stats.boxplot(column='total_pnl', by='frequency_segment', ax=axes[0, 1])
axes[0, 1].set_title('Total PnL by Trading Frequency')
axes[0, 1].set_ylabel('Total PnL (USD)')

trader_stats.boxplot(column='total_pnl', by='consistency_segment', ax=axes[1, 0])
axes[1, 0].set_title('Total PnL by Consistency')
axes[1, 0].set_ylabel('Total PnL (USD)')

segment_win_rates = trader_stats.groupby(['leverage_segment', 'frequency_segment'])['avg_win_rate'].mean().unstack()
segment_win_rates.plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Win Rate by Leverage and Frequency')
axes[1, 1].set_ylabel('Average Win Rate')
axes[1, 1].legend(title='Frequency')

plt.tight_layout()
plt.savefig('outputs/segment_analysis.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/segment_analysis.png")

# Additional Insights
print("\n" + "-" * 80)
print("ADDITIONAL INSIGHTS")
print("-" * 80)

# Correlation analysis
correlation_data = daily_metrics[['fg_value', 'daily_pnl', 'win_rate', 'num_trades', 'leverage_proxy']].corr()
print("\nCorrelation Matrix:")
print(correlation_data)

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation: Fear/Greed Index vs Trading Metrics')
plt.tight_layout()
plt.savefig('outputs/correlation_heatmap.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/correlation_heatmap.png")

# Time series
sentiment_timeline = daily_metrics.groupby(['date', 'sentiment_group'])['daily_pnl'].sum().reset_index()
sentiment_timeline = sentiment_timeline.pivot(index='date', columns='sentiment_group', values='daily_pnl').fillna(0)
sentiment_timeline_cumsum = sentiment_timeline.cumsum()

plt.figure(figsize=(15, 6))
for col in sentiment_timeline_cumsum.columns:
    plt.plot(sentiment_timeline_cumsum.index, sentiment_timeline_cumsum[col], label=col, linewidth=2)
plt.title('Cumulative PnL Over Time by Sentiment Period')
plt.xlabel('Date')
plt.ylabel('Cumulative PnL (USD)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/cumulative_pnl_timeline.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/cumulative_pnl_timeline.png")

# Segment-sentiment performance
pivot_data = daily_metrics.groupby(['leverage_segment', 'sentiment_group'])['daily_pnl'].mean().unstack()
plt.figure(figsize=(10, 6))
pivot_data.plot(kind='bar', ax=plt.gca())
plt.title('Average Daily PnL by Leverage Segment and Sentiment')
plt.ylabel('Average Daily PnL (USD)')
plt.xlabel('Leverage Segment')
plt.legend(title='Sentiment')
plt.tight_layout()
plt.savefig('outputs/segment_sentiment_performance.png', dpi=300, bbox_inches='tight')
print("\nChart saved: outputs/segment_sentiment_performance.png")

## PART C: ACTIONABLE STRATEGIES

In [ ]:
print("\n" + "=" * 80)
print("PART C: ACTIONABLE STRATEGY RECOMMENDATIONS")
print("=" * 80)

# Calculate specific metrics for recommendations
high_lev_fear = daily_metrics[(daily_metrics['leverage_segment'] == 'High Leverage') & 
                              (daily_metrics['sentiment_group'] == 'Fear')]['daily_pnl'].mean()
high_lev_greed = daily_metrics[(daily_metrics['leverage_segment'] == 'High Leverage') & 
                               (daily_metrics['sentiment_group'] == 'Greed')]['daily_pnl'].mean()

print(f"\nSTRATEGY 1: Leverage Management")
print(f"High leverage traders during Fear: ${high_lev_fear:.2f} avg daily PnL")
print(f"High leverage traders during Greed: ${high_lev_greed:.2f} avg daily PnL")
print(f"Recommendation: {'Reduce leverage during Fear periods' if high_lev_fear < high_lev_greed else 'Maintain leverage strategy'}")

freq_fear = daily_metrics[(daily_metrics['frequency_segment'] == 'Frequent') & 
                         (daily_metrics['sentiment_group'] == 'Fear')]['daily_pnl'].mean()
infreq_fear = daily_metrics[(daily_metrics['frequency_segment'] == 'Infrequent') & 
                           (daily_metrics['sentiment_group'] == 'Fear')]['daily_pnl'].mean()

print(f"\nSTRATEGY 2: Trading Frequency During Fear")
print(f"Frequent traders during Fear: ${freq_fear:.2f} avg daily PnL")
print(f"Infrequent traders during Fear: ${infreq_fear:.2f} avg daily PnL")
print(f"Recommendation: {'Reduce trade frequency during Fear' if freq_fear < infreq_fear else 'Maintain or increase frequency during Fear'}")

# Export results
print("\n" + "-" * 80)
print("EXPORTING RESULTS")
print("-" * 80)

summary_stats = {
    'Total Traders': trader_stats['Account'].nunique(),
    'Total Trading Days': daily_metrics['date'].nunique(),
    'Avg Daily PnL (Fear)': daily_metrics[daily_metrics['sentiment_group'] == 'Fear']['daily_pnl'].mean(),
    'Avg Daily PnL (Greed)': daily_metrics[daily_metrics['sentiment_group'] == 'Greed']['daily_pnl'].mean(),
    'Win Rate (Fear)': daily_metrics[daily_metrics['sentiment_group'] == 'Fear']['win_rate'].mean(),
    'Win Rate (Greed)': daily_metrics[daily_metrics['sentiment_group'] == 'Greed']['win_rate'].mean(),
}

summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('outputs/analysis_summary.csv', index=False)
print("\nSummary statistics saved: outputs/analysis_summary.csv")

daily_metrics.to_csv('outputs/daily_metrics_processed.csv', index=False)
trader_stats.to_csv('outputs/trader_segments.csv', index=False)
print("Processed data saved:")
print("- outputs/daily_metrics_processed.csv")
print("- outputs/trader_segments.csv")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE!")
print("=" * 80)
print("\nGenerated files:")
print("- outputs/performance_by_sentiment.png")
print("- outputs/behavior_by_sentiment.png")
print("- outputs/segment_analysis.png")
print("- outputs/correlation_heatmap.png")
print("- outputs/cumulative_pnl_timeline.png")
print("- outputs/segment_sentiment_performance.png")
print("- outputs/analysis_summary.csv")
print("- outputs/daily_metrics_processed.csv")
print("- outputs/trader_segments.csv")
print("\nSee FINDINGS.md for detailed analysis and recommendations.")